In [2]:
#coding=utf-8
import requests
from urllib import parse
from openpyxl import load_workbook
from openpyxl.utils import get_column_interval
import pandas as pd
import numpy as np
import re
import sys
import time

def load_workbook_range(range_string, ws):
    col_start, col_end = re.findall("[A-Z]+", range_string)

    data_rows = []
    for row in ws[range_string]:
        data_rows.append([cell.value for cell in row])

    return pd.DataFrame(data_rows, columns=get_column_interval(col_start, col_end))

wb = load_workbook(filename='ss_num.xlsx')
#for ws in wb.worksheets:
ws = wb.active    
df = load_workbook_range('A2:A10', ws)
print(df)
sl = [3,3,3,3,3,3,3,3,3] #ssNum
ul = [77,114,80,54,135,66,179,124,19] #suNum

list1 = ['0', '1', '2']
list2 = ['01','02','03','04','05','06','07','08','09','10','11','12']

count = 0
index = 0
for id, item in enumerate(df.A):
    index = index + 1
    error = 0
    name = item 
    for l1 in list1:
        for l2 in list2:         
            print(parse.quote(name)+l1+'旬'+l2)    
            try:
                sname = parse.quote(name)
                url = 'http://xx.xx.xx.xx/%A6%AF%B3%F8.asp?stat='+l1+'&YY=2020&MM='+l2+'&DD=' 
                my_referer = 'http://xx.xx.xx.xx/na_main.asp?ssNum=0'+str(sl[id])+'&suNum='+str(ul[id])+'&suName='+sname            
                accept = 'text/html, application/xhtml+xml, image/jxr, */*'
                agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'           
                header = {
                    'user-agent': agent,
                    'referrer': my_referer,
                    'Accept': accept,
                    'Accept-Encoding': 'gzip, deflate',
                    'Accept-Language': 'zh-Hant-TW,zh-Hant;q=0.5',
                    'Content-Type': 'application/x-www-form-urlencoded',
                    'Cache-Control': 'private',
                    'Connection': 'keep-alive'
                } 
                #get cookie
                gc_url = 'http://10.241.6.101/exeform.asp' 
                session = requests.Session()  
                response = session.get(gc_url, headers = header, allow_redirects=False)
                d = session.cookies.get_dict()
                #d = {'a': 'Apple', 'b': 'ball', 'c': 'cat'}
                cookies = ";".join(("{}={}".format(*i) for i in d.items()))  # a=Apple;c=cat;b=ball
                print(cookies)
                header = {
                    'user-agent': agent,
                    'referrer': my_referer,
                    'Accept': accept,
                    'Accept-Encoding': 'gzip, deflate',
                    'Accept-Language': 'zh-Hant-TW,zh-Hant;q=0.5',
                    'Cookie': cookies  #renew every time            
                } 
                req = requests.get(url, headers = header)
            except Exception as e:
                print(e)
                print(item+l1+'旬_2020'+l2+'error')
                error = 1
                break #break list2
            
            print(req.status_code)
            url_content = req.content
            csv_file = open(parse.unquote(sname)+'DS'+l1+'2020'+l2+'ss=0'+str(sl[id])+'&su='+str(ul[id])+'_load.html','wb')
            csv_file.write(url_content)
            csv_file.close()
            if l2 == '03':  #try only one csv file
                break
            delays = [2, 3, 4]
            delay = np.random.choice(delays)
            time.sleep(delay)
    if index == 1:
        break


FileNotFoundError: [Errno 2] No such file or directory: 'ss_num.xlsx'